# Running PySpark with Cassandra in Jupyter

Blog post is available at: https://medium.com/@amirziai/running-pyspark-with-cassandra-in-jupyter-2bf5e95c319#.1zhdl0ovk

### Pre-requisite

### Create a bash script to run Jupyter

In [118]:
%%writefile start_jupyter_cassandra.sh

#!/bin/bash

cd $SPARK_HOME/python/pyspark/
export IPYTHON_OPTS="notebook"
pyspark --jars /home/ec2-user/pyspark-cassandra/target/scala-2.10/pyspark-cassandra-assembly-0.2.7.jar --driver-class-path /home/ec2-user/pyspark-cassandra/target/scala-2.10/pyspark-cassandra-assembly-0.2.7.jar --conf spark.cassandra.connection.host=10.0.0.0 &> /dev/null &

Writing start_jupyter_cassandra.sh


#### Make the script executable

In [122]:
!chmod a+x start_jupyter_cassandra.sh

#### Now run the script in your terminal

Make sure you run this first so the JAR and connection host are enabled
```
./start_jupyer_cassandra.sh
```

### Start

In [1]:
sc

In [2]:
import pyspark_cassandra

In [3]:
sc

In [4]:
def get_table(key_space, table):
    return sc.cassandraTable(key_space, table)

In [5]:
table = get_table('key_sapce', 'table')

In [7]:
%%time
table.cassandraCount()

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 2.28 s


142102L

In [8]:
%%time
table.count()

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 4.59 s


142102

In [16]:
def check_bmi(x, threshold):
    try:
        return float(x) >= threshold
    except:
        return False
    
def check_height(x, threshold):
    try:
        return float(x.split()[0]) >= threshold
    except:
        return False

In [11]:
results = table.select('bmi', 'height')
results.take(5)

[Row(bmi=u'20.89', height=u'5 ft 3 in'),
 Row(bmi=u'25.28', height=u'5 ft 8 in'),
 Row(bmi=None, height=None),
 Row(bmi=u'18.41', height=u'5 ft 6 in'),
 Row(bmi=u'27.04', height=u'5 ft 2.5 in')]

In [13]:
filtered = results.filter(lambda x: check_bmi(x['bmi'], 35) and check_height(x['height'], 5))
filtered.take(5)

[Row(bmi=u'35.13', height=u'5 ft 8 in'),
 Row(bmi=u'40.00', height=u'5 ft 2 in'),
 Row(bmi=u'55.04', height=u'5 ft 3 in'),
 Row(bmi=u'47.02', height=u'5 ft 10 in'),
 Row(bmi=u'47.02', height=u'5 ft 10 in')]

In [24]:
%%time
def get_height(x):
    try:
        return float(x.split()[0])
    except:
        return 0

print filtered.map(lambda x: (get_height(x['height']), 1)) \
.reduceByKey(lambda a, b: a + b) \
.collect()

[(6.0, 1626), (5.0, 13995), (7.0, 3)]
CPU times: user 0 ns, sys: 8 ms, total: 8 ms
Wall time: 2.5 s
